In [15]:
#função de propagação linear
#  net/2

funcao.prop <- function(net){
   # return(1/1+exp(-net))
    return(net/2)
}
der.funcao.prop <- function(i){
    return(1/2)
    #return(i*(1-i))
}


In [10]:
# Arquitetura da Rede MLP
arquitetura <- function(num.entrada, num.oculta, num.saida, funcao.prop, der.funcao.prop){
    arq <- list()
    arq$num.entrada <- num.entrada
    arq$num.oculta <- num.oculta
    arq$num.saida <- num.saida
    arq$funcao.prop <- funcao.prop
    arq$der.funcao.prop <- der.funcao.prop
    
    #cria matriz para valores desejados
    arq$desejado <- diag(1,num.saida,num.saida)
    
    #definir os pesos da camada oculta (num.entrada+1) por causa do Bias
    num.pesos.entrada.oculta <- (num.entrada+1)*num.oculta
    
    #inicializar pesos da camada oculta
    arq$oculta <- matrix(runif(min=-1,max=1,num.pesos.entrada.oculta), 
                         nrow=num.oculta, ncol=num.entrada+1)
    
    #definir os pesos da camada de saída
    num.pesos.oculta.saida <- num.saida * (num.oculta+1)

    #inicializar pesos da camada de saida
    arq$saida <- matrix(runif(min=-1,max=1,num.pesos.oculta.saida), 
                         nrow=num.saida, ncol=num.oculta+1)

    return(arq)
}


In [11]:
#Propagação

mlp.propagacao <- function(arq, exemplo){
    
    # propagar da entrada -> camada oculta
    net.entrada.oculta <- arq$oculta %*% as.numeric(c(exemplo,1))
    
    i.entrada.oculta <- arq$funcao.prop(net.entrada.oculta)
    
    # propagar da oculta -> camada de saída
    net.oculta.saida <- arq$saida %*% as.numeric(c(i.entrada.oculta,1))
    
    i.oculta.saida <- arq$funcao.prop(net.oculta.saida)

    # Resultado

    res <- list()
    res$net.entrada.oculta <- net.entrada.oculta
    res$i.entrada.oculta <- i.entrada.oculta
    res$net.oculta.saida <- net.oculta.saida
    res$i.oculta.saida <- i.oculta.saida 
    
    return(res)

}


In [12]:
# backpropagation

mlp.retropropagacao <- function(arq, dados_treino, n, limiar, max.epocas){
    
erroQuadratico <- limiar*2
epocas <- 0
    
while (erroQuadratico > limiar && epocas < max.epocas){    
    
    erroQuadratico <- 0
    
    #embaralhar os dados de treino
    set.seed(100)
    indice <- sample(1:nrow(dados_treino))
    dados <- dados_treino[indice,]
    
    
    #treino
    for( i in 1:nrow(dados)){
        
        #pega a entrada
        x.entrada <- dados[i, 1:arq$num.entrada]
        #pega a saída
        x.saida <- as.numeric(dados[i,ncol(dados)])
                
        #saída da rede 
        res <- mlp.propagacao(arq, x.entrada)
        y<- res$i.oculta.saida
        
        #Calcular o erro
        erro <- arq$desejado[x.saida,] - y
    
        #Acumula o erro da rede de cada exemplo
        erroQuadratico <- erroQuadratico + (sum(erro^2)/2)
        
        #Calcula o GErro para a camada de saída
        grad.saida <- erro * arq$der.funcao.prop(y)
        pesos.saida <- arq$saida[,1:arq$num.oculta]
                
        #Calcular o GErro para a camada oculta
        grad.oculta <- as.numeric(arq$der.funcao.prop(res$i.entrada.oculta)) * 
                       (t(grad.saida) %*% pesos.saida)
        
        # ajustar os pesos
        #Saida
        arq$saida <- arq$saida + n * (grad.saida %*% c(res$i.entrada.oculta, 1)) #acrtescenta o bias
        
        #Oculta
        arq$oculta <- arq$oculta + n * (t(grad.oculta) %*% as.numeric(c(x.entrada,1))) #acrescenta o bias
    }
    
    erroQuadratico <- erroQuadratico / nrow(dados)
    epocas <- epocas +1
   # if(epocas %% 2 == 0)
        cat("Época -> ", epocas, " -- Erro quadrático médio -> ", erroQuadratico," \n") 
}

retorno <- list()
retorno$arq <- arq
retorno$epocas <- epocas

return(retorno)
}


In [5]:
normaliza <- function(atributo){
   num <- atributo - min(atributo)    
   intervalo <- max(atributo) - min(atributo)
   resultado <- num/intervalo
   
    return(resultado)
}

In [17]:
#montar a arquitetura
arq <- arquitetura(4,6,3,funcao.prop, der.funcao.prop)

#separar dados entre treino e teste
dados <- iris

set.seed(132)
indice.amostras <- sample(2,nrow(dados), replace=TRUE, prob=c(0.7,0.3))

dados_treino <- dados[indice.amostras==1,]
dados_teste <- dados[indice.amostras==2,]




In [ ]:
modelo <- mlp.retropropagacao(arq, dados_treino, 0.05, 0.001, 10000)

In [32]:
dados_teste

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
2,4.9,3.0,1.4,0.2,setosa
7,4.6,3.4,1.4,0.3,setosa
10,4.9,3.1,1.5,0.1,setosa
12,4.8,3.4,1.6,0.2,setosa
17,5.4,3.9,1.3,0.4,setosa
19,5.7,3.8,1.7,0.3,setosa
20,5.1,3.8,1.5,0.3,setosa
24,5.1,3.3,1.7,0.5,setosa
25,4.8,3.4,1.9,0.2,setosa


In [39]:
amostra <- dados_teste[49,-5]
resposta <- dados_teste[49,5]

In [40]:
amostra
resposta

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
,<dbl>,<dbl>,<dbl>,<dbl>
145,6.7,3.3,5.7,2.5


[1] virginica
Levels: setosa versicolor virginica

In [41]:
mlp.propagacao(modelo$arq, amostra)

-4.600361
-3.081459
1.695618
4.589166
-1.924954
-1.182323
-2.3001804
-1.5407294
0.8478092
2.2945832
-0.9624772


In [38]:
dados

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
<dbl>,<dbl>,<dbl>,<dbl>,<fct>
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa
4.6,3.4,1.4,0.3,setosa
5.0,3.4,1.5,0.2,setosa
4.4,2.9,1.4,0.2,setosa
